In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
def table(x,y,ex=None,ey=None,bx=True,by=True):
    if bx==True and by==True:
        dic={'x':x,'ex':ex,'y':y,'ey':ey}
    elif bx==False:
        dic={'x':x,'y':y,'ey':ey}
    elif by==False:
        dic={'x':x,'ex':ex,'y':y}
    else:
        dic={'x':x,'y':y}
    tab=pd.DataFrame(dic)
    return (tab)

In [3]:
def round_sig(x, sig=2):
    from math import floor, log10
    return round(x, sig-int(floor(log10(abs(x))))-1)
rs=np.vectorize(round_sig)


def round_err_based(value,error):
    from math import log10,ceil,floor
    v=(floor(log10(abs(value))))
    e=(floor(log10(abs(error))))
    sig=(v-e)+1
    return round_sig(value,sig)
reb=np.vectorize(round_err_based)

def max_err_prop_prod(x,ex,y,ey):
    return x*ey+ex*y
def max_err_prop_div(x,ex,y,ey):
    return 1/y*ex+x/y**2*ey
mepp=np.vectorize(max_err_prop_prod)
mepd=np.vectorize(max_err_prop_div)

def lin_fit_sigma_fit(x,y):
    N = len(x)
    x2 = x**2
    xy = x*y
    X = sum(x)
    X2 = sum(x2)
    XY = sum(xy)
    Y = sum(y)
    A = np.array([[X2, X],[X, N]])
    B = np.array([[XY],[Y]])
    sol = np.linalg.solve(A, B)
    a = sol[0, 0]
    b = sol[1, 0]
    Delta = N*X2-X**2
    y_fit = a*x+b
    sigma_fit = np.sqrt(sum((y_fit-y)**2)/(N-1))
    sigmaa = sigma_fit * np.sqrt(N/Delta)
    sigmab = sigma_fit * np.sqrt(X2/Delta)
    covab = -X/Delta * sigma_fit**2
    rho = covab/(sigmaa*sigmab)
    res = np.array([a, sigmaa, b, sigmab, rho])
    return res

def plot(x,y,xe,ye,fit=False,cs=5):
    fig, ax = plt.subplots(figsize=(10,10), dpi=100)
    ax.errorbar(x,y,xerr=xe,yerr=ye,linestyle='',capsize=cs,fmt='.',label='data')
    if fit==True:
        a,sigmaa,b,sigmab,rho=lin_fit_sigma_fit(x,y)
        def lin(x,a,b):
            return a*x+b
        m,M=x.min(),x.max()
        ax.plot([m,M],[lin(m,a,b),lin(M,a,b)],zorder=100,color='orange',label=r'fit $a\cdot x+b$:'+'\n'+r'a={:.4f}+/-{:.4f}'.format(a,sigmaa)+'\n'+'b={:.4f}+/-{:.4f}'.format(b,sigmab)+'\n'+'corr.:{:.1f}'.format(rho))
    ax.legend()
    plt.show()

In [4]:
def max_err_prop_prod(x,ex,y,ey):
    return x*ey+ex*y
def max_err_prop_div(x,ex,y,ey):
    return 1/y*ex+x/y**2*ey
mepp=np.vectorize(max_err_prop_prod)
mepd=np.vectorize(max_err_prop_div)

In [5]:
def QY(qm,ym,qM,yM):
    try:
        n=len(qm)
    except:
        n=1
    errq=rs((qM-qm)/2,1)
    erry=rs((yM-ym)/2,1)
    y=reb((yM+yM)/2,erry)
    q=reb((qM+qm)/2,errq)
    dic={'q':q,'errq':errq,'y':y,'erry':erry}
    tab=pd.DataFrame(dic,index=np.linspace(1,n,n))
    return tab

In [ ]:
y1=2
fname='alphaR.txt'
data1=np.loadtxt(fname,dtype=float,comments='#')
p,Qm,ym,QM,yM=data1
errb=0.5
errr=0.1
qm,qM=Qm-p,QM-p
QY1=QY(qm,ym,qM,yM)
q=np.array(QY1.q)
errq=np.array(QY1.errq)
y=np.array(QY1.y)
erry=np.array(QY1.erry)
alphaR=y/q
erraR=mepd(y,erry,q,errq)
erraR=rs(erraR,1)
alphaR=reb(alphaR,erraR)
print('alphaR = ({}+/-{}) rad'.format(alphaR,erraR))
QY1

In [ ]:
pcryst=20
plens=10
fname2='beta.txt'
data2=np.loadtxt(fname2, dtype=float,comments='#')
P,Qm,ym,QM,yM=data2[:,0],data2[:,1],data2[:,2],data2[:,3],data2[:,4]
qm,qM=Qm-pcryst,QM-pcryst
QY2=QY(qm,ym,qM,yM)
q=np.array(QY2.q)
errq=np.array(QY2.errq)
y=np.array(QY2.y)
erry=np.array(QY2.erry)
beta=y/q
errbeta=mepd(y,erry,q,errq)
errbeta=rs(errbeta,1)
beta=reb(beta,errbeta)
table(beta,errbeta,bx=False,by=False)
QY2

In [ ]:
G=beta/alphaR
errG=rs(mepd(beta,errbeta,alphaR,erraR),2)
G=rs(G,2)
table(G,errG,bx=False,by=False)

In [ ]:
plot(P, G,[0.5]*len(P),errG,True)
table(P, G,[0.5]*len(P),errG)

In [ ]:
d0=25
f=10
print('G mean = {:.1f} +/- {:.1f}'.format(G.mean(), errG.mean()))
print('attesa={}'.format(d0/f))

In [ ]:
z1=0.4
fname3='alpha2.txt'
data3=np.loadtxt(fname3,dtype=float,comments='#')
p,Qm,zm,QM,zM=data3
errb=0.5
errr=0.1
qm,qM=Qm-p,QM-p
q=(qM+qm)/2
errq=(qM-qm)/2
z=(zm+zM)/2
errz=(zM-zm)/2
alpha2=z/q
erra2=mepd(z,errz,q,errq)
erra2=rs(erra2,1)
alpha2=reb(alpha2,erra2)
print('alpha2 = ({}+/-{}) rad'.format(alpha2,erra2))

In [6]:
z1=1
fname3='alpha2.txt'
data3=np.loadtxt(fname3,dtype=float,comments='#')
p,Qm,zm,QM,zM=data3
errb=0.5
errr=0.1
qm,qM=Qm-p,QM-p
QZ=QY(qm,zm,qM,zM)
q=np.array(QZ.q)
errq=np.array(QZ.errq)
z=np.array(QZ.y)
errz=np.array(QZ.erry)
alpha2=z/q
erra2=mepd(z,errz,q,errq)
erra2=rs(erra2,1)
alpha2=reb(alpha2,erra2)
print('alpha2 = ({}+/-{}) rad'.format(alpha2,erra2))
errQr=rs(errq,1)
Qr=reb(q,errQr)
print('Qr = ({}+/-{})cm'.format(Qr,errQr))
QZ

alpha2 = ([0.023]+/-[0.003]) rad
Qr = ([39.]+/-[1.])cm


,q,errq,y,erry
1.0,39.0,1.0,0.9,0.1


In [7]:
Z2=float(input('Inserire misura Z2: '))
errZ2=float(input('Inserire misura Z1: '))
errbeta2=rs(mepd(Z2,errZ2,Qr,errQr),1)
beta2=reb(Z2/Qr,errbeta2)
print('beta2=({}+/-{})'.format(beta2,errbeta2))
errGv=rs(mepd(beta2,errbeta2,alpha2,erra2),1)
Gv=reb(beta2/alpha2,errGv)
print('Gv = ({} +/- {})'.format(Gv, errGv))

Inserire misura Z2: 9
Inserire misura Z1: 0.1
beta2=([0.231]+/-[0.008])
Gv = ([10.] +/- [2.])


In [ ]:
fob=5
foc=10
Delta=20
d0=25
Gvexp=d0*Delta/(fob*foc)
print('Gv attesa = {}'.format(Gvexp))